virtual environment 'sent.analyzer'

    conda install -n sent.analyzer spacy
    python -m spacy download en_core_web_sm

# Quick Tutorial

### Tokenizing
Tokenization is the process of breaking down chunks of text into smaller pieces. spaCy comes with a default processing pipeline that begins with tokenization, making this process a snap. In spaCy, you can do either sentence tokenization or word tokenization:

    Word tokenization breaks text down into individual words.
    Sentence tokenization breaks text down into individual sentences.

In [1]:
import spacy

ModuleNotFoundError: No module named 'spacy'